# Interpreting Linear Regression

In this notebook we will continue with our car price prediction example and explore the methods we use to interpret and evaluate the results of our linear regression model. We will begin by reproducing the results from the previous notebook.

As usual, we first import the required packages and dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plot
import statsmodels.api as stats
import numpy as np

In [ ]:
carprice_df = pd.read_csv('CarPrice_Assignment.csv')

We then remove the independent variables with too many unique categories or have high correlations with other independent variables.

In [ ]:
carprice_df = carprice_df

Next we one-hot-encode the categorical variables so each category in a categorical variable becomes its own binary column - this converts categorical variables to numeric. We also remove more highly correlated variables after including the one-hot-encoded variables.

In [ ]:
dummy = 

In [ ]:
carprice_df = pd.concat([carprice_df.select_dtypes(exclude='object'), dummy], axis=1)

In [ ]:
carprice_df = carprice_df.drop(columns=['compressionratio', 'drivewheel_fwd', 'enginetype_rotor', 'fuelsystem_4bbl', 'fuelsystem_idi'])

We then split our data into train and test sets, and add our constant column.

In [ ]:
train_df=carprice_df.sample(frac=0.7, random_state=101) 
test_df=carprice_df.drop(train_df.index)

In [ ]:
Y_train = 
X_train = 

We then fit our model to the training data

In [ ]:
model_carprice = 
results_carprice = model_carprice.fit()

In [ ]:
Y_test = test_df.price
test_df = stats.add_constant(test_df)
X_test = test_df[X_train.columns]

We finally produce our test set predictions



In [ ]:
test_predictions = results_carprice.predict(X_test)

# p-Values and Coefficients

### p-Values

In [ ]:
print(results_carprice.summary())

### Re-training the Model

In [ ]:
Y_train_new = train_df.price
X_train_new = stats.add_constant(train_df[['enginesize', 
                                    'stroke', 
                                    'peakrpm', 
                                    'fueltype_gas', 
                                    'carbody_hardtop', 
                                    'carbody_hatchback', 
                                    'enginelocation_rear',
                                    'enginetype_ohc',
                                    'cylindernumber_five',
                                    'cylindernumber_four',
                                    'cylindernumber_six']])

In [ ]:
model_carprice_new = stats.OLS(Y_train_new, X_train_new)
results_carprice_new = model_carprice_new.fit()

In [ ]:
print(results_carprice_new.summary())

All of our independent variables now have p-values below 0.05 (except const which we are not concerned whether this is zero or not) and we have reduced the complexity of our model by reducing the number of variables.

In [ ]:
Y_test_new = test_df.price
X_test_new = test_df[X_train_new.columns]

In [ ]:
test_predictions_new = results_carprice_new.predict(X_test_new)

In [ ]:
plot.scatter(test_predictions_new, Y_test_new)
plot.plot([5000, 50000], [5000, 50000], c='k', ls='--')
plot.xlabel('Predicted Price [$]')
plot.ylabel('Observed Price [$]')
plot.show()

### Coefficients

The coefficients tell us how much the target variable changes for a one unit change in the corresponding independent variable when all the other independent variables are held fixed. We can get a measure of which independent variable contributes the most change to the target variable if we first scale all the independent variables to remove the units. This is known as standardization as we discussed in the theory lesson.

In [ ]:
X_train_scale = ((X_train_new - X_train_new.mean())/X_train_new.std()).drop(columns=['const'])

In [ ]:
X_train_scale

In [ ]:
X_train_scale

As we can see the independent variables now all have mean zero and a standard deviation of one and will will therefore be able to compare coefficient values. We will now refit the model. Scikit-Learn also contains a function StandardScaler() which will perform this standardization.

In [ ]:
X_train_scale = stats.add_constant(X_train_scale)

In [ ]:
model_carprice_scale = stats.OLS(Y_train_new, X_train_scale)
results_carprice_scale = model_carprice_scale.fit()

In [ ]:
print(results_carprice_scale.summary())

In [ ]:
results_carprice_scale.params.drop(index=['const']).abs().sort_values(ascending=False).plot.bar()

# Residuals and Residual Plots

In [ ]:
results_carprice_new.resid

We can then plot these residuals against our predicted values for the training data and observe if we see any non-random pattern in the data.

In [ ]:
plot.scatter(results_carprice_new.fittedvalues, results_carprice_new.resid)
plot.plot([5000,50000], [0,0], c='k', ls='--')

# Evaluating Linear Regression

statsmodels allows us to easily calculate the metrics we have discussed (mean square error, root mean square error, mean absolute error,  R2  and adjusted  R2 ) We can calculate these for either/both the training data and the test data.

In [ ]:
train_mse = stats.tools.eval_measures.mse(Y_train_new, results_carprice_new.fittedvalues)
print('The training dataset mean square error is {}'.format(train_mse.round(1)))

In [ ]:
train_rmse = stats.tools.eval_measures.rmse(Y_train_new, results_carprice_new.fittedvalues)
print('The training dataset root mean square error is {}'.format(train_rmse.round(1)))

In [ ]:
train_mae = stats.tools.eval_measures.meanabs(Y_train_new, results_carprice_new.fittedvalues)
print('The training dataset mean absolute error is {}'.format(train_mae.round(1)))

In [ ]:
train_r2 = results_carprice_new.rsquared
print('The training dataset coefficient of determination is {}'.format(train_r2.round(3)))

In [ ]:
train_r2_adj = results_carprice_new.rsquared_adj
print('The training dataset adjusted coefficient of determination is {}'.format(train_r2_adj.round(3)))

### Scikit-Learn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
Y = carprice_df.price
X = carprice_df[['enginesize', 
                 'stroke', 
                 'peakrpm', 
                 'fueltype_gas', 
                 'carbody_hardtop', 
                 'carbody_hatchback', 
                 'enginelocation_rear',
                 'enginetype_ohc',
                 'cylindernumber_five',
                 'cylindernumber_four',
                 'cylindernumber_six']]

In [ ]:
sk_X_train, sk_X_test, sk_Y_train, sk_Y_test = train_test_split(X, Y, test_size=0.3, random_state=97)

In [ ]:
regressor = LinearRegression()  
regressor.fit(sk_X_train, sk_Y_train)

LinearRegression()

In [ ]:
sk_intercept_carprice = regressor.intercept_
sk_engsize_coeffs = regressor.coef_
sk_ssr_carprice = np.sum((sk_Y_train-regressor.predict(sk_X_train))**2)

In [ ]:
sk_train_predictions = regressor.predict(sk_X_train)
sk_test_predictions = regressor.predict(sk_X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
sk_train_mse = mean_squared_error(sk_Y_train, sk_train_predictions)
print('The training dataset mean square error is {}'.format(sk_train_mse.round(1)))

The training dataset mean square error is 6501288.6


In [ ]:
sk_train_rmse = np.sqrt(sk_train_mse)
print('The training dataset root mean square error is {}'.format(sk_train_rmse.round(1)))

The training dataset root mean square error is 2549.8


In [ ]:
sk_train_mae = mean_absolute_error(sk_Y_train, sk_train_predictions)
print('The training dataset mean absolute error is {}'.format(sk_train_mae.round(1)))

The training dataset mean absolute error is 1904.1


In [ ]:
sk_train_r2 = r2_score(sk_Y_train, sk_train_predictions)
print('The training dataset coefficient of determination is {}'.format(sk_train_r2.round(3)))

The training dataset coefficient of determination is 0.872


In [ ]:
n = sk_X_train.shape[0]
p = sk_X_train.shape[1]
sk_train_r2_adj = 1-(1-sk_train_r2)*(n-1)/(n-p-1)
print('The training dataset adjusted coefficient of determination is {}'.format(sk_train_r2_adj.round(3)))

The training dataset adjusted coefficient of determination is 0.861


### Test Data

In [ ]:
sk_test_r2 = r2_score(sk_Y_test, sk_test_predictions)
print('The test dataset coefficient of determination is {}'.format(sk_test_r2.round(3)))

The test dataset coefficient of determination is 0.837
